# Lecture 10
## Symbolic Non Linear Optimization (Part 2)
## Date: 21.11

In [4]:
using LinearAlgebra
using SymPy
using Plots

In [11]:
x,y,z = Sym("x,y,z")

(x, y, z)

## Recap exercises

### Ex 1 

Given $h(x,y)$ as:

$$ h(x,y) = e^{-x^2}(2xy-y^2) $$

Test the algorithm you defined in the previous lecture to get the critical points and their nature. 

#### Solution

In [8]:
h(x,y) = exp(-x^2)*(2*x*y - y^2)

h (generic function with 1 method)

In [9]:
#Compute gradient
∇h = [diff(h(x,y),i) for i in free_symbols(h(x,y))];

#Get the critical points
critical_points = solve(∇h);

#Compute the Hessian
Hh = hessian(h(x,y),[x,y]);

In [10]:
for q in critical_points 
    global H_in_P = [subs(Hh[i,j],q) for i in 1:2, j in 1:2]
    println("The point")
    println(q)
    
    if prod([i>0  for i in eigvals(H_in_P)])
        println("The point is a local minimum")
    elseif prod([i<0  for i in eigvals(H_in_P)])
        println("The point is a local maximum")    
    else 
        println("The point is a saddle")
    end
    
    println("")
    
end

The point
Dict{Any,Any}(x=>-1,y=>-1)
The point is a local maximum

The point
Dict{Any,Any}(x=>0,y=>0)
The point is a saddle

The point
Dict{Any,Any}(x=>1,y=>1)
The point is a local maximum



### Ex 2 

Given $h(x,y)$ as:

$$ h(x,y) = e^x(2x^2-xy+y^2) $$

Test the algorithm you defined in the previous lecture to get the critical points and their nature. 

#### Solution

In [59]:
f = x^2 + 2*y^2 + 3*z^2 + 2*x*y + 2*x*z

   2                      2      2
- x  - 2⋅x⋅y - 2⋅x⋅z - 2⋅y  - 3⋅z 

In [60]:
#Compute gradient
∇f = [diff(f,i) for i in free_symbols(f)];

#Get the critical points
critical_points = solve(∇f,dict = true);

#Compute the Hessian
Hf = hessian(f,free_symbols(f));

In [62]:
for q in critical_points 
    H_in_P = [subs(Hf[i,j],q) for i in 1:length(free_symbols(f)), j in 1:length(free_symbols(f))]
    
    println("The point")
    println(q)
    
    principal_minor = [det(H_in_P[1:i,1:i]) for i in 1:size(H_in_P)[1]]
    
    principal_minor_sign_flipped = principal_minor .* [(-1)^i for i in 1:length(principal_minor)]
    
    if prod([i>0  for i in principal_minor])
        println("The point is a local minimum")
    elseif prod([i>0  for i in principal_minor_sign_flipped])
        println("The point is a local maximum")    
    else 
        println("The point is a saddle")
    end
    
    println("")
    
end

The point
Dict{Any,Any}(x=>0,y=>0,z=>0)
The point is a local maximum



## Constrained optimization

### Ex 1
Given the function $f(x,y,z)$:

$$
x+2z \text{ s.t. } \begin{cases} x^2+y^2+z^2 = 5 \end{cases}
$$

Develop an algorithm that finds the critical points and their nature of the above constrained maximization problem.

In [156]:
f = x + 2z;

constraints = [x^2 + y^2 + z^2 - 5];

lagrange_multipliers = symbols("λ1:"*string(length(constraint)+1));
variables = setdiff(free_symbols(Λ),lagrange_multipliers);

n = length(variables);
m = length(lagrange_multipliers);

last_minors = n-m;

In [140]:
Λ = f -sum(lagrange_multipliers.*constraints) 

             ⎛ 2    2    2    ⎞
x + 2⋅z - λ1⋅⎝x  + y  + z  - 5⎠

In [141]:
#Compute gradient
∇Λ = [diff(Λ,i) for i in free_symbols(Λ)];

#Get the critical points
critical_points = solve(∇Λ,dict = true);

In [142]:
#Compute the Hessian
HΛ = hessian(Λ,variables)

#Compute partial derivatives of constraints
∇constraints = [diff(constraints[k],i) for i in variables, k in 1:length(constraints)]

#Evaluate the leading part of the bordered Hessian
leading = zeros(length(constraints),length(constraints));

#Get the bordered Hessian
bordered_hessian = hcat(vcat(leading,∇constraints),vcat((∇constraints.T),HΛ))

4×4 Array{Sym,2}:
 0.0    2*x    2*y    2*z
 2*x  -2*λ1      0      0
 2*y      0  -2*λ1      0
 2*z      0      0  -2*λ1

In [174]:
for q in critical_points 
    
    global BH_in_P = [subs(bordered_hessian[i,j],q) for i in 1:size(bordered_hessian)[1], j in 1:size(bordered_hessian)[1]]
    
    println("The point")
    println(q)
    
    principal_minor = [det(BH_in_P[1:i,1:i]) for i in (size(BH_in_P)[1]-last_minors + 1):size(BH_in_P)[1]] .* (-1)^m   
    
    principal_minor_sign_flipped = [det(BH_in_P[1:i,1:i]) for i in (size(BH_in_P)[1]-last_minors + 1):size(BH_in_P)[1]] .* [(-1)^(n+i-length(principal_minor)) for i in 1:length(principal_minor)]
    
    if prod([i>0  for i in principal_minor])
        println("The point is a local minimum")
    elseif prod([i>0  for i in principal_minor_sign_flipped])
        println("The point is a local maximum")    
    else 
        println("The point is a saddle")
    end
    
    println("")
    
end

The point
Dict{Any,Any}(x=>-1,y=>0,z=>-2,λ1=>-1/2)
The point is a local minimum

The point
Dict{Any,Any}(x=>1,y=>0,z=>2,λ1=>1/2)
The point is a local maximum



In [170]:
BH_in_P

4×4 Array{Sym,2}:
 0.0   2   0   4
   2  -1   0   0
   0   0  -1   0
   4   0   0  -1

In [175]:
principal_minor = [det(BH_in_P[1:i,1:i]) for i in (size(BH_in_P)[1]-last_minors + 1):size(BH_in_P)[1]]

2-element Array{Sym,1}:
   4
 -20

### Ex 2
Given the function $f(x,y,z)$:

$$
y \text{ s.t. } \begin{cases} x^2+y^2+z^2 = 2 \\ y=z \end{cases}
$$

Test the algorithm you developed.

In [180]:
f = y;

constraints = [x^2+y^2+z^2 - 2, y-z];

lagrange_multipliers = symbols("λ1:"*string(length(constraints)+1));
variables = setdiff(free_symbols(Λ),lagrange_multipliers);

n = length(variables);
m = length(lagrange_multipliers);

last_minors = n-m;

In [182]:
Λ = f -sum(lagrange_multipliers.*constraints) 

       ⎛ 2    2    2    ⎞             
y - λ1⋅⎝x  + y  + z  - 2⎠ - λ2⋅(y - z)

In [183]:
#Compute gradient
∇Λ = [diff(Λ,i) for i in free_symbols(Λ)];

#Get the critical points
critical_points = solve(∇Λ,dict = true);

In [184]:
#Compute the Hessian
HΛ = hessian(Λ,variables)

#Compute partial derivatives of constraints
∇constraints = [diff(constraints[k],i) for i in variables, k in 1:length(constraints)]

#Evaluate the leading part of the bordered Hessian
leading = zeros(length(constraints),length(constraints));

#Get the bordered Hessian
bordered_hessian = hcat(vcat(leading,∇constraints),vcat((∇constraints.T),HΛ))

5×5 Array{Sym,2}:
 0.0  0.0    2*x    2*y    2*z
 0.0  0.0      0      1     -1
 2*x    0  -2*λ1      0      0
 2*y    1      0  -2*λ1      0
 2*z   -1      0      0  -2*λ1

In [185]:
for q in critical_points 
    
    global BH_in_P = [subs(bordered_hessian[i,j],q) for i in 1:size(bordered_hessian)[1], j in 1:size(bordered_hessian)[1]]
    
    println("The point")
    println(q)
    
    principal_minor = [det(BH_in_P[1:i,1:i]) for i in (size(BH_in_P)[1]-last_minors + 1):size(BH_in_P)[1]] .* (-1)^m   
    
    principal_minor_sign_flipped = [det(BH_in_P[1:i,1:i]) for i in (size(BH_in_P)[1]-last_minors + 1):size(BH_in_P)[1]] .* [(-1)^(n+i-length(principal_minor)) for i in 1:length(principal_minor)]
    
    if prod([i>0  for i in principal_minor])
        println("The point is a local minimum")
    elseif prod([i>0  for i in principal_minor_sign_flipped])
        println("The point is a local maximum")    
    else 
        println("The point is a saddle")
    end
    
    println("")
    
end

The point
Dict{Any,Any}(x=>0,y=>-1,λ2=>1/2,z=>-1,λ1=>-1/4)
The point is a local minimum

The point
Dict{Any,Any}(x=>0,y=>1,λ2=>1/2,z=>1,λ1=>1/4)
The point is a local maximum

